In [ ]:
import torch
torch.set_float32_matmul_precision("medium")


In [ ]:
!pip install -q transformers accelerate bitsandbytes einops

In [ ]:
from huggingface_hub import login

login(token="...")  # Replace with your real token

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

tokenizer.pad_token = tokenizer.eos_token


# Load model in 8-bit to fit into Colab GPU
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_8bit=True
)

# Set to eval mode
model.eval()


In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

class ToxicityClassifierPipeline:
    def __init__(self, binary_model_path, fine_model_path, device="cuda"):
        self.device = torch.device(device if torch.cuda.is_available() else "cpu")

        self.tokenizer_binary = AutoTokenizer.from_pretrained(binary_model_path)
        self.model_binary = AutoModelForSequenceClassification.from_pretrained(binary_model_path).to(self.device)

        self.tokenizer_fine = AutoTokenizer.from_pretrained(fine_model_path)
        self.model_fine = AutoModelForSequenceClassification.from_pretrained(fine_model_path).to(self.device)

        self.label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
        self.label_to_explanation = {
            "toxic": "This sentence contains general toxic language.",
            "severe_toxic": "This sentence contains extreme hostility or verbal abuse.",
            "obscene": "This sentence contains obscene or vulgar language.",
            "threat": "This sentence contains a threat or implied violence.",
            "insult": "This sentence includes personal insults or demeaning language.",
            "identity_hate": "This sentence attacks someone based on identity (e.g. race, gender, religion)."
        }

    def __call__(self, comment, threshold=0.5):
        inputs = self.tokenizer_binary(comment, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            outputs = self.model_binary(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            toxic_prob = probs[:, 1].item()

        binary_pred = "toxic" if toxic_prob >= threshold else "non-toxic"

        if binary_pred == "non-toxic":
            return {"binary": binary_pred, "subtypes": None, "toxic_prob": toxic_prob, "explanation": None}

        inputs = self.tokenizer_fine(comment, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            outputs = self.model_fine(**inputs)
            fine_probs = torch.sigmoid(outputs.logits).cpu().numpy()[0]

        subtypes = {
            label: round(float(prob), 2)
            for label, prob in zip(self.label_cols, fine_probs)
            if prob >= threshold
        }

        explanation_parts = [self.label_to_explanation[label] for label in subtypes]
        explanation = " ".join(explanation_parts) if explanation_parts else None

        return {
            "binary": binary_pred,
            "subtypes": subtypes,
            "toxic_prob": toxic_prob,
            "explanation": explanation
        }

In [ ]:
pipeline = ToxicityClassifierPipeline(
    binary_model_path="/content/drive/MyDrive/...",
    fine_model_path="/content/drive/MyDrive/..."
)

with Stage 1 :

1.Install Dependencies
2.Load the LLaMA Model
3.Load Stage 1 Classifier
4.Define the Prompt

In [ ]:
def build_llama_prompt_with_explanation(toxic, explanation):
    explanation = explanation.strip() if explanation else "This sentence contains toxic language."
    return f"""[INST] Rewrite the following sentence in one sentence to make it polite and non-toxic while keeping its meaning. The explanation highlights why the sentence is considered toxic:
{toxic}
Explanation: {explanation}
[/INST]"""



In [ ]:
import re
import torch

def prompt_and_generate_with_explanation(toxic, explanation):
    prompt = f"""[INST] Rewrite the following sentence in one sentence to make it polite and non-toxic while keeping its meaning. The explanation highlights why the sentence is considered toxic:
{toxic}
Explanation: {explanation}
[/INST]"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256,
        return_attention_mask=True
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # --- Clean output ---
    # Remove prompt echo
    decoded = decoded.split("[/INST]")[-1].strip()

    # Remove common prefixes
    decoded = re.sub(
        r"(?i)^(rewritten|rephrased|revised|restated)?\s*sentence\s*[:\-]?\s*", "", decoded
    )
    decoded = re.sub(r"(?i)^(final answer|answer|note|output)[:\-]?\s*", "", decoded)
    decoded = re.sub(r"(?i)^here('?s| is)?\s+(a|the)?\s*rewritten\s+version\s*(of (the )?sentence)?[:\-]?\s*", "", decoded)
    decoded = re.sub(r"(?i)^##\s*", "", decoded)

    # Extract only the first sentence
    match = re.match(r"^(.*?[.!?])(\s|$)", decoded)
    detoxified = match.group(1).strip() if match else decoded.strip()

    return detoxified

 6. Combined Prompt + Stage 1 Explanation with a fallback (This sentence contains toxic language.)

In [ ]:
def prompt_and_generate_with_explanation(toxic_sentence):
    stage1 = pipeline(toxic_sentence)
    explanation = stage1.get("explanation", "This sentence contains toxic language.")
    prompt = build_llama_prompt_with_explanation(toxic_sentence, explanation)
    return generate_response(prompt)


for one sentence :

In [ ]:
import re
import torch

toxic = "you are an idot."
explanation = "This sentence contains an insult."

# ✅ Build prompt
prompt = build_llama_prompt_with_explanation(toxic, explanation)

# ✅ Tokenize with attention mask
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=256,
    return_attention_mask=True
).to("cuda")

# ✅ Generate detoxified output
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
    )

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ Remove prompt echo
decoded = decoded.split("[/INST]")[-1].strip()

# ✅ Strip intro phrases
decoded = re.sub(
    r"(?i)^(rewritten|rephrased|revised|restated)?\s*sentence\s*[:\-]?\s*", "", decoded
)
decoded = re.sub(r"(?i)^(final answer|answer|note|output)[:\-]?\s*", "", decoded)
decoded = re.sub(r"(?i)^here('?s| is)?\s+(a|the)?\s*rewritten\s+version\s*(of (the )?sentence)?[:\-]?\s*", "", decoded)
decoded = re.sub(r"(?i)^##\s*", "", decoded)

# ✅ Extract just the first complete sentence
match = re.match(r"^(.*?[.!?])(\s|$)", decoded)
detoxified = match.group(1).strip() if match else decoded.strip()

print("Detoxified:", detoxified)

for test set:

In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("/content/drive/MyDrive/...")
df["generated_output"] = df["toxic_sentence"].progress_apply(prompt_and_generate_with_explanation)

df.to_csv("/content/drive/MyDrive/...", index=False)
print("Saved output with Stage 1 prompts.")


In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/...')

evaluation:

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/...")

In [ ]:
!pip install bert-score evaluate sacrebleu
from bert_score import score as bert_score

P, R, F1 = bert_score(
    cands=df["generated_output"].tolist(),
    refs=df["neutral_reference"].tolist(),
    lang="en"
)

print(f"BERTScore F1: {F1.mean():.4f}")

In [ ]:
!pip install sentence-transformers sacrebleu

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/...")

# Rename columns to standard names for code consistency
df.rename(columns={
    "toxic_sentence": "input",
    "generated_output": "prediction",
    "neutral_reference": "reference"
}, inplace=True)

In [ ]:
def is_toxic(text):
    result = pipeline(text)
    return result["binary"] == "toxic"

df["STA"] = [0 if is_toxic(pred) else 1 for pred in df["prediction"]]

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load multilingual sentence similarity model
labse = SentenceTransformer("sentence-transformers/LaBSE")

emb_input = labse.encode(df["input"].tolist(), convert_to_tensor=True)
emb_pred = labse.encode(df["prediction"].tolist(), convert_to_tensor=True)

similarities = util.cos_sim(emb_input, emb_pred).diagonal().tolist()
df["SIM"] = similarities

In [ ]:
from sacrebleu.metrics import CHRF

chrf = CHRF()
chrf_scores = [
    chrf.sentence_score(pred, [ref]).score / 100
    for pred, ref in zip(df["prediction"], df["reference"])
]
df["CHRF"] = chrf_scores

In [ ]:
import numpy as np

# Final J-score is the average of STA, SIM, and CHRF
df["J-score"] = (df["STA"] + df["SIM"] + df["CHRF"]) / 3

# Print final average
print(f"✅ J-score (mean over all examples): {df['J-score'].mean():.4f}")

In [ ]:
df.to_csv("/content/...", index=False)

In [ ]:
from google.colab import files
files.download('/content/...')

In [ ]:
import numpy as np

# Metric means from previously computed columns
df["J-score"] = (df["STA"] + df["SIM"] + df["CHRF"]) / 3

bert_f1 = F1.mean().item()                     # BERTScore F1
chrf_score = df["CHRF"].mean()                # CHRF
sta_score = df["STA"].mean()                  # STA
sim_score = df["SIM"].mean()                  # SIM
j_score = df["J-score"].mean()                # J-score

# Calculate overall average metrics
average_scores = {
    "Metric": ["BERTScore F1", "STA", "SIM", "CHRF", "Average J-score"],
    "Score": [
        bert_f1,
        sta_score,
        sim_score,
        chrf_score,
        j_score
    ]
}

# Convert to DataFrame and export
summary_df = pd.DataFrame(average_scores)
summary_df.to_csv("/content/drive/MyDrive/...", index=False)

In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/...')